# KNN

In [1]:
import numpy as np
import pandas as pd
from sklearn.impute import KNNImputer
from fancyimpute import IterativeImputer  # pip install fancyimpute
import os
import math
import tqdm
import sklearn.neighbors._base
import sys
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
from missingpy import MissForest # 

(CVXPY) Jun 23 11:25:26 AM: Encountered unexpected exception importing solver CVXOPT:
ImportError('DLL load failed while importing base: Le module spécifié est introuvable.')
(CVXPY) Jun 23 11:25:26 AM: Encountered unexpected exception importing solver GLPK:
ImportError('DLL load failed while importing base: Le module spécifié est introuvable.')
(CVXPY) Jun 23 11:25:26 AM: Encountered unexpected exception importing solver GLPK_MI:
ImportError('DLL load failed while importing base: Le module spécifié est introuvable.')


In [52]:
# pip install missingpy

In [2]:
# Move up 2 directories
data_directory = '../..' 

# Load the CSV files
asthma_df = pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv'))

In [4]:
# In the asthma dataset there seems to be a problem with HR00 to HR23
# The heart rate goes above a million sometimes
# It seems like the first couple digits correspond with the expected heart rate.
# 2 cases: HR below 100 and HR above 100

# Loop through each column in the dataframe
def preprocess(asthma_df):
    for col in asthma_df.loc[:, "HR00":"HR23"]:
        for i in range(len(asthma_df[col])):
            if (asthma_df.loc[i, col] > 1000):
                # Take first 2 digits of float
                first_2_digits = str(asthma_df.loc[i, col])[:2]
                temp_number = float(first_2_digits)
                # If heart rate lower than 20 it means it should be greater than 100 (assuming heart rates < 200)
                if temp_number < 20:
                    val = str(asthma_df.loc[i, col])[:3] + '.' + str(asthma_df.loc[i, col])[3]
                    asthma_df.loc[i, col] = float(val)
                else:
                    val = str(asthma_df.loc[i, col])[:2] + '.' + str(asthma_df.loc[i, col])[2]
                    asthma_df.loc[i, col] = float(val)
                pass
    return asthma_df


In [5]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    random_index = np.random.randint(0, len(df))
    random_column = columns[np.random.randint(0, len(columns))]

    df.loc[random_index, random_column]

    while pd.isna(df.loc[random_index, random_column]):
        random_index = np.random.randint(0, len(df))
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column]

In [6]:
asthma_tmp = preprocess(pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv')))
asthma_tmp.to_csv(('asthma_tmp.csv'), index=False)

In [ ]:
# # Play with the number of neighbors
# # Look at the kids with the fewest missing data
# difference = []
# for i in tqdm.tqdm(range(1000)):
#     asthma_df = pd.read_csv('asthma_tmp.csv')
#     random_index = get_random_index(asthma_df, ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
#                 "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
#     tmp = asthma_df.loc[random_index[0], random_index[1]]
#     asthma_df.loc[random_index[0], random_index[1]] = np.nan
#     imputer = KNNImputer(n_neighbors = 5)          # n_neighbors=2
#     asthma_df.loc[:, "HR00":"HR23"] = imputer.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
#     # get the absolute value of the difference between the original value and the imputed value
#     difference.append(abs(tmp - asthma_df.loc[random_index[0], random_index[1]]))



# print(difference)

In [ ]:
# print(difference)

In [7]:
adress = []
for i in range(5000):
    adress.append(get_random_index(asthma_df, ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                                               "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"]))

# TRY

In [ ]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data

# TODO: read the csv all the time.

differenceKNN = []
differenceMICE = []
differenceMissForest = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp.csv')
    tmp = asthma_df.loc[random_index[0], random_index[1]]
    asthma_df.loc[random_index[0], random_index[1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    # asthma_df.loc[:, "HR00":"HR23"] = imputerKNN.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    asthma_KNN = (imputerKNN.fit_transform(asthma_df.loc[:, "HR00":"HR23"]))
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - asthma_KNN.loc[random_index[0], random_index[1]]))

    asthma_df = pd.read_csv('asthma_tmp.csv')

    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMICE.append(abs(tmp - asthma_MICE.loc[random_index[0], random_index[1]]))

    


c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4952/5000 [1:46:36<01:13,  1.53s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4953/5000 [1:46:38<01:09,  1.48s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4954/5000 [1:46:39<01:05,  1.43s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
 99%|█████████▉| 4955/5000 [1:46:40<01:03,  1.41s/it]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [Iterat

In [ ]:
print(differenceKNN)
print(differenceMICE)

print(sum(differenceKNN)/len(differenceKNN))
print(sum(differenceMICE)/len(differenceMICE))

[8.620000000000005, 14.780000000000001, 1.0800000000000125, 1.279999999999987, 12.279999999999987, 16.900000000000006, 3.8800000000000097, 5.299999999999997, 9.490000000000009, 10.799999999999997, 18.099999999999994, 22.059999999999988, 11.199999999999989, 18.820000000000007, 20.580000000000013, 18.08, 3.559999999999988, 0.519999999999996, 20.950000000000003, 16.519999999999996, 9.170000000000016, 26.890000000000015, 5.8799999999999955, 9.699999999999989, 0.6799999999999926, 6.739999999999995, 4.159999999999982, 14.239999999999995, 2.839999999999989, 11.330000000000013, 2.1200000000000045, 28.97, 11.849999999999994, 2.8699999999999903, 20.230000000000004, 2.0999999999999943, 0.9599999999999937, 0.9799999999999969, 1.0, 9.090000000000003, 8.469999999999999, 7.6299999999999955, 1.8200000000000074, 2.780000000000001, 5.180000000000007, 18.47999999999999, 0.6200000000000045, 6.019999999999982, 7.640000000000015, 5.950000000000003, 6.400000000000006, 11.659999999999997, 1.7800000000000011, 

7.7043589054054
6.565687295977342

In [8]:
for random_index in tqdm.tqdm(adress): 
    asthma_df = pd.read_csv('asthma_tmp.csv')  
    tmp = asthma_df.loc[random_index[0], random_index[1]]
    asthma_df.loc[random_index[0], random_index[1]] = np.nan 
    imputerMiss = MissForest(criterion = 'squared_error', verbose=0)
    asthma_MISS = imputerMiss.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    asthma_MISS = pd.DataFrame(asthma_MISS, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMissForest.append(abs(tmp - asthma_MISS.loc[random_index[0], random_index[1]]))

  0%|          | 0/5000 [00:00<?, ?it/s]

Iteration: 0
Iteration: 1
Iteration: 2
Iteration: 3
Iteration: 4


  0%|          | 0/5000 [01:09<?, ?it/s]

Iteration: 5


NameError: name 'differenceMissForest' is not defined

In [ ]:
MissforestResult = [8.849500000000006, 8.525500000000065, 1.5380000000000251, 2.1424999999999272, 12.227999999999966, 19.800500000000028, 0.8365000000000293, 5.993000000000038, 6.760500000000064, 2.506500000000031, 10.60350000000004, 20.401499999999984, 17.742500000000007, 18.80450000000002, 24.747000000000014, 10.105500000000006, 0.9700000000000415, 3.221500000000006, 11.695100000000025, 13.90000000000002, 4.294499999999999, 22.733500000000006, 9.317500000000038, 6.351999999999961, 3.154500000000013, 4.5594999999999715, 5.7270000000000465, 15.317999999999984, 2.7725000000000364, 3.0135000000000076, 9.003500000000017, 21.52049999999997, 0.973500000000044, 5.0449999999999875, 14.147999999999968, 5.020999999999987, 2.4985000000000213, 1.6170000000000186, 1.89650000000006, 10.496000000000038, 7.934999999999945, 5.907999999999973, 3.4500000000000597, 2.1599999999999966, 1.9080000000000297, 20.7055, 1.0184999999999462, 0.9110000000000582, 7.376000000000047, 3.452000000000055, 0.5734999999999815, 6.993500000000026, 0.9170000000000442, 5.46850000000002, 12.538399999999967, 9.792500000000004, 1.6625000000000085, 6.314000000000021, 15.41150000000006, 9.87949999999995, 2.2804999999999893, 3.8420000000000414, 6.862000000000009, 2.786000000000044, 2.242000000000033, 8.525000000000063, 7.6555000000000035, 0.27400000000002933, 7.187499999999972, 7.337000000000032, 10.239000000000033, 11.228499999999983, 3.226499999999973, 2.1125000000000256, 9.596499999999992, 3.4510000000000076, 7.389000000000024, 0.7134999999999394, 18.493499999999983, 1.767500000000041, 7.626999999999953, 2.442499999999953, 14.399999999999949, 0.7835000000000321, 9.005999999999986, 5.681499999999971, 1.5380000000000535, 19.037999999999997, 1.1520000000000579, 14.43049999999998, 10.819500000000005, 6.657299999999992, 2.2035000000000053, 5.368500000000068, 6.2880000000000535, 0.34600000000002495, 9.37049999999995, 1.7074999999999534, 1.9280000000000115, 11.136999999999972, 2.3460000000000036, 12.166999999999973, 9.36499999999998, 3.6110000000000184, 10.368499999999997, 0.24750000000001648, 18.619999999999933, 0.8035000000000423, 4.484500000000068, 3.7485000000000355, 3.8670000000000897, 3.0599999999999454, 3.6739999999999355, 0.41349999999997067, 1.3759999999999906, 1.612000000000009, 4.6490000000000435, 5.099499999999949, 0.6125000000000256, 14.36499999999998, 2.225999999999999, 3.6730000000000587, 2.083999999999989, 7.39549999999997, 2.2700000000000387, 4.4755000000000535, 5.993399999999966, 0.1599999999999966, 2.431500000000085, 3.666500000000056, 4.549500000000023, 6.100500000000011, 9.69000000000004, 29.36400000000002, 1.1970000000000312, 3.687499999999986, 19.119499999999988, 5.3160000000000025, 6.2475000000000165, 12.467500000000058, 10.276500000000027, 3.813499999999962, 0.7352000000000345, 4.105999999999938, 1.0844999999999487, 4.093499999999949, 5.833499999999987, 12.833499999999987, 11.292500000000032, 7.028500000000037, 9.115500000000011, 0.8340000000000742, 4.940500000000014, 3.146000000000001, 1.1945000000000476, 13.289499999999933, 18.191000000000017, 0.17200000000002547, 10.960500000000039, 16.077499999999958, 8.631, 9.26700000000001, 1.2519999999999527, 8.376499999999993, 3.282499999999999, 1.4245000000000374, 25.340000000000003, 13.144499999999965, 4.223999999999975, 2.8869999999999294, 1.3425000000000722, 7.610500000000016, 13.32250000000002, 6.032999999999987, 6.357500000000044, 1.4015000000000413, 8.829499999999996, 1.2135000000000318, 0.522500000000008, 8.418500000000009, 15.378500000000003, 9.004999999999981, 3.9830000000000396, 16.94900000000002, 5.084500000000048, 3.4080000000000723, 4.251500000000021, 33.51649999999994, 7.968000000000018, 11.940500000000014, 4.556000000000012, 18.12649999999998, 15.434499999999971, 3.016500000000036, 25.18649999999998, 3.6980000000000075, 10.32399999999997, 4.6910000000000025, 3.2944999999999283, 3.4135000000000133, 3.9969999999999857, 8.606500000000068, 0.19300000000001205, 0.8994999999999607, 4.487499999999969, 5.077500000000029, 17.335500000000025, 0.3985000000000696, 2.5699999999999505, 5.142000000000024, 0.4294999999999476, 2.2620000000000573, 10.302000000000078, 2.8474999999999824, 1.8724999999999739, 5.709500000000048, 1.073000000000036, 5.414500000000018, 0.5199999999999818, 9.189000000000036, 7.5530000000000115, 10.599000000000032, 11.68799999999996, 11.523499999999999, 2.8805000000000263, 4.724999999999994, 8.762999999999963, 14.629000000000048, 12.462399999999931, 10.080500000000029, 3.5710000000000264, 11.050500000000014, 4.679999999999964, 7.653000000000034, 4.614499999999964, 5.30199999999995, 10.154000000000025, 5.123500000000021, 7.10199999999999, 7.21700000000007, 1.5120000000000573, 3.850000000000037, 8.263499999999965, 17.135500000000064, 0.17749999999993804, 12.527500000000003, 3.9290000000000305, 1.0459999999999212, 2.103500000000011, 8.910499999999999, 1.7215000000000487, 24.730500000000035, 9.481499999999997, 5.808000000000035, 0.6609999999999587, 1.8705000000000638, 3.284499999999966, 3.3034999999999854, 0.3480000000000132, 22.933499999999967, 3.0620000000000545, 4.969999999999985, 2.0840000000000316, 6.564000000000092, 11.80850000000001, 2.824000000000069, 0.7484999999999502, 4.782500000000084, 8.220000000000027, 1.8140000000000498, 14.985499999999973, 6.367000000000033, 13.848500000000058, 2.2169999999999703, 6.069500000000019, 0.8600000000000421, 4.09699999999998, 8.82750000000005, 5.672500000000014, 9.84399999999998, 6.889999999999972, 6.320999999999998, 8.215999999999951, 10.665499999999966, 5.584500000000006, 4.886999999999944, 6.402499999999989, 7.911000000000001, 7.300499999999971, 4.815000000000055, 4.81399999999995, 12.436999999999912, 12.291500000000013, 0.32299999999995066, 14.336500000000015, 3.4079999999999444, 25.876499999999993, 6.737000000000037, 2.90799999999993, 0.7970000000000113, 1.61850000000004, 6.152000000000072, 10.954999999999998, 3.889499999999984, 6.6984999999999815, 4.754500000000064, 10.853500000000025, 2.8095, 1.8660000000000565, 1.664000000000044, 9.076500000000038, 0.8190000000000168, 9.421999999999954, 4.129999999999967, 9.282000000000039, 7.9140000000000015, 2.7784999999999656, 3.702000000000055, 0.06549999999994327, 1.0795000000000243, 4.860000000000028, 14.40449999999997, 1.2070000000000363, 3.398500000000027, 4.672999999999945, 1.7579999999999956, 4.902499999999904, 2.6759999999999735, 9.927499999999952, 6.8579999999999615, 2.213500000000039, 2.086999999999975, 11.341500000000053, 1.964500000000001, 4.3589999999999804, 0.14300000000004331, 8.131000000000043, 9.898999999999958, 0.5020000000000522, 3.0224999999999653, 6.201499999999982, 1.504499999999993, 3.3195000000000903, 2.7379999999999285, 3.990600000000029, 3.9320000000000306, 3.5704999999999814, 4.1860000000000355, 0.7894999999999897, 14.582499999999996, 5.633999999999958, 2.0564999999999287, 12.838499999999954, 5.903499999999994, 5.386500000000012, 12.623499999999964, 2.51400000000001, 1.202999999999932, 3.5989999999999753, 3.35750000000003, 2.9924999999999784, 2.018999999999977, 18.223000000000027, 5.082000000000065]

print(np.var(MissforestResult))

32.044052275547216


In [ ]:

print(sum(differenceMissForest)/len(differenceMissForest))

6.643018956043962


6.643018956043962

## Compute the kids with the fewest missing values

In [12]:
def extractChild(df):

    # Create a new database per children and save it into a list of all the same subject numbers.
    df_children = []    # List of all the children
    
    # Loop through all the subject numbers
    for subject in df['SubjectNr'].unique():
        df_children.append(df[df['SubjectNr'] == subject])
    
    return df_children

In [13]:
# Move up 2 directories
data_directory = '../..' 

# Load the CSV files
asthma_df = pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv'))

asthma_df_children = extractChild(asthma_df)

counter = []
#for every child compute the number of Nan in the HR columns
for i in range(len(asthma_df_children)):
    counter.append((i, asthma_df_children[i].loc[:, "HR00":"HR23"].isna().sum().sum()))

print(counter)

# sort the counter list by the number of Nan
counter.sort(key=lambda x: x[1])

print(counter)

[(0, 24), (1, 91), (2, 72), (3, 15), (4, 23), (5, 16), (6, 76), (7, 61), (8, 25), (9, 53), (10, 85), (11, 273), (12, 56), (13, 230), (14, 42), (15, 613), (16, 108), (17, 36), (18, 16), (19, 13), (20, 18), (21, 100), (22, 98), (23, 108), (24, 42), (25, 39), (26, 40), (27, 163), (28, 47), (29, 420), (30, 109), (31, 21), (32, 38), (33, 36), (34, 37), (35, 460), (36, 24), (37, 23), (38, 22), (39, 93), (40, 36), (41, 47), (42, 345), (43, 28), (44, 27), (45, 28), (46, 431), (47, 52), (48, 156), (49, 130), (50, 29), (51, 39), (52, 76), (53, 411), (54, 92), (55, 225), (56, 44), (57, 190), (58, 41), (59, 284), (60, 19), (61, 19), (62, 72), (63, 38), (64, 20), (65, 54), (66, 43), (67, 69), (68, 42), (69, 26), (70, 82), (71, 26), (72, 124), (73, 67), (74, 38), (75, 40), (76, 28), (77, 27), (78, 60), (79, 213), (80, 131), (81, 42), (82, 102), (83, 32), (84, 32), (85, 293), (86, 117), (87, 59), (88, 46), (89, 20)]
[(19, 13), (3, 15), (5, 16), (18, 16), (20, 18), (60, 19), (61, 19), (64, 20), (89, 2

In [14]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    tmp = df.index
    number = np.random.randint(0, len(tmp))
    random_index = tmp[number]
    random_column = columns[np.random.randint(0, len(columns))]

    while pd.isna(df.loc[random_index, random_column]):
        tmp = df.index
        random_index = tmp[np.random.randint(0, len(tmp))]
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column, number]

In [15]:
interestingKid = [19,3,5,18,20,60,61,64,89,31,38,4,37,0,36,8,69,71,44,77,43,45,76,50]


randomKid = []
for i in range(2500):
    # get random element of interestingKid
    randomKid.append(interestingKid[np.random.randint(0, len(interestingKid))])

adress = []
for kid in randomKid:
    adress.append((kid, get_random_index(asthma_df_children[kid], ["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                                               "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])))

print(adress)

[(69, [2011, 'HR03', 9]), (19, [573, 'HR22', 22]), (0, [24, 'HR21', 24]), (31, [923, 'HR20', 24]), (71, [2072, 'HR07', 0]), (20, [602, 'HR22', 22]), (61, [1797, 'HR15', 28]), (37, [1099, 'HR00', 26]), (31, [924, 'HR00', 25]), (76, [2223, 'HR07', 17]), (64, [1868, 'HR08', 12]), (69, [2027, 'HR16', 25]), (18, [523, 'HR02', 1]), (45, [1313, 'HR04', 0]), (50, [1471, 'HR22', 21]), (37, [1089, 'HR11', 16]), (0, [12, 'HR01', 12]), (50, [1464, 'HR04', 14]), (8, [244, 'HR02', 12]), (18, [534, 'HR11', 12]), (61, [1777, 'HR20', 8]), (5, [169, 'HR22', 24]), (77, [2248, 'HR01', 13]), (19, [554, 'HR20', 3]), (71, [2075, 'HR01', 15]), (5, [169, 'HR21', 24]), (61, [1772, 'HR16', 3]), (3, [103, 'HR03', 16]), (4, [122, 'HR21', 6]), (50, [1467, 'HR10', 17]), (61, [1774, 'HR03', 5]), (0, [8, 'HR02', 8]), (8, [248, 'HR12', 16]), (37, [1097, 'HR10', 24]), (38, [1120, 'HR20', 18]), (0, [6, 'HR21', 6]), (44, [1277, 'HR12', 1]), (20, [583, 'HR17', 3]), (60, [1756, 'HR16', 0]), (44, [1283, 'HR07', 7]), (89, [26

In [ ]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data

# TODO: read the csv all the time.

differenceKNN = []
differenceMICE = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    # asthma_df.loc[:, "HR00":"HR23"] = imputerKNN.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    asthma_KNN = (imputerKNN.fit_transform(asthma_df_children[random_index[0]].loc[:, "HR00":"HR23"]))
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - asthma_KNN.loc[random_index[1][2], random_index[1][1]]))

    asthma_df = pd.read_csv('asthma_tmp.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df_children[random_index[0]].loc[:, "HR00":"HR23"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMICE.append(abs(tmp - asthma_MICE.loc[random_index[1][2], random_index[1][1]]))

  0%|          | 3/2500 [00:01<15:17,  2.72it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  0%|          | 10/2500 [00:03<14:36,  2.84it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 17/2500 [00:05<12:03,  3.43it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 22/2500 [00:07<11:41,  3.53it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(
  1%|          | 23/2500 [00:07<13:01,  3.17it/s]c:\Users\Admin\anaconda3\lib\site-packages\sklearn\impute\_iterative.py:

In [ ]:
print(sum(differenceKNN)/len(differenceKNN))
print(sum(differenceMICE)/len(differenceMICE))

# the variance
print(np.var(differenceKNN))
print(np.var(differenceMICE))

7.153252000000006
6.897327812555437
41.669131304496
40.72816928317416


In [ ]:
differenceMissForest = []

for random_index in tqdm.tqdm(adress[:300]): 
    asthma_df = pd.read_csv('asthma_tmp.csv') 
    asthma_df_children = extractChild(asthma_df) 
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan 
    imputerMiss = MissForest(criterion = 'squared_error', verbose=0)
    asthma_MISS = imputerMiss.fit_transform(asthma_df_children[random_index[0]].loc[:, "HR00":"HR23"])
    asthma_MISS = pd.DataFrame(asthma_MISS, columns=["HR00", "HR01", "HR02", "HR03", "HR04", "HR05", "HR06", "HR07", "HR08", "HR09", "HR10", "HR11", "HR12", "HR13",
                "HR14", "HR15", "HR16", "HR17", "HR18", "HR19", "HR20", "HR21", "HR22", "HR23"])
    differenceMissForest.append(abs(tmp - asthma_MISS.loc[random_index[1][2], random_index[1][1]]))

In [17]:
print(sum(differenceMissForest)/len(differenceMissForest))
print(np.var(differenceMissForest))

7.0514066666666695
36.0566725929555


# STEPS

Evaluation of the steps with child

In [5]:
def extractChild(df):

    # Create a new database per children and save it into a list of all the same subject numbers.
    df_children = []    # List of all the children
    
    # Loop through all the subject numbers
    for subject in df['SubjectNr'].unique():
        df_children.append(df[df['SubjectNr'] == subject])
    
    return df_children

In [6]:
def preprocess(asthma_df):
    asthma_df.loc[:, "steps00":"steps10"] = asthma_df.loc[:, "steps00":"steps10"].fillna(0.0)
    asthma_df.loc[:, "steps22":"steps23"] = asthma_df.loc[:, "steps22":"steps23"].fillna(0.0)
    return asthma_df


In [7]:
data_directory = '../..' 

asthma_tmp = preprocess(pd.read_csv(os.path.join(data_directory, 'Data', 'astma.csv')))
asthma_tmp.to_csv(('asthma_tmp_steps.csv'), index=False)

In [8]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    tmp = df.index
    number = np.random.randint(0, len(tmp))
    random_index = tmp[number]
    random_column = columns[np.random.randint(0, len(columns))]

    while pd.isna(df.loc[random_index, random_column]):
        tmp = df.index
        random_index = tmp[np.random.randint(0, len(tmp))]
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column, number]

In [9]:
interestingKid = [19,3,5,18,20,60,61,64,89,31,38,4,37,0,36,8,69,71,44,77,43,45,76,50]

asthma_df = pd.read_csv('asthma_tmp_steps.csv')
asthma_df_children = extractChild(asthma_df)

randomKid = []
for i in range(2500):
    # get random element of interestingKid
    randomKid.append(interestingKid[np.random.randint(0, len(interestingKid))])

adress = []
for kid in randomKid:
    adress.append((kid, get_random_index(asthma_df_children[kid], ["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"])))

print(adress)

[(4, [134, 'steps19', 18]), (37, [1099, 'steps14', 26]), (20, [586, 'steps15', 6]), (69, [2015, 'steps19', 13]), (89, [2585, 'steps15', 2]), (50, [1472, 'steps19', 22]), (50, [1450, 'steps17', 0]), (89, [2605, 'steps13', 22]), (20, [592, 'steps21', 12]), (64, [1869, 'steps17', 13]), (69, [2015, 'steps12', 13]), (77, [2260, 'steps14', 25]), (44, [1278, 'steps17', 2]), (60, [1746, 'steps13', 6]), (37, [1084, 'steps21', 11]), (50, [1464, 'steps16', 14]), (8, [235, 'steps16', 3]), (18, [528, 'steps11', 6]), (76, [2223, 'steps14', 17]), (89, [2591, 'steps14', 8]), (3, [90, 'steps21', 3]), (89, [2608, 'steps19', 25]), (31, [926, 'steps12', 27]), (5, [160, 'steps20', 15]), (64, [1856, 'steps19', 0]), (43, [1251, 'steps15', 4]), (64, [1857, 'steps14', 1]), (44, [1295, 'steps13', 19]), (19, [565, 'steps20', 14]), (77, [2252, 'steps15', 17]), (61, [1771, 'steps17', 2]), (38, [1108, 'steps18', 6]), (19, [552, 'steps18', 1]), (5, [147, 'steps20', 10]), (89, [2589, 'steps18', 6]), (37, [1085, 'step

In [ ]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data

# TODO: read the csv all the time.

differenceKNN = []
differenceMICE = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp_steps.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    asthma_KNN = imputerKNN.fit_transform(asthma_df_children[random_index[0]].loc[:, "steps11":"steps21"])
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=[["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"]])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - (asthma_KNN.loc[random_index[1][2], random_index[1][1]])[0]))

    asthma_df = pd.read_csv('asthma_tmp_steps.csv')
    asthma_df_children = extractChild(asthma_df)
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan
    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df_children[random_index[0]].loc[:, "steps11":"steps21"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=[["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"]])
    differenceMICE.append(abs(tmp - (asthma_MICE.loc[random_index[1][2], random_index[1][1]])[0]))

In [ ]:
print(differenceMICE[0])

26.009351025580372


In [20]:
print(sum(differenceKNN)/len(differenceKNN))
print(sum(differenceMICE)/len(differenceMICE))

# the variance
print(np.var(differenceKNN))
print(np.var(differenceMICE))

352.661040000001
358.2866777002531
173628.5712021184
161053.59673862148


In [ ]:
differenceMissForest = []

for random_index in tqdm.tqdm(adress[:300]): 
    asthma_df = pd.read_csv('asthma_tmp.csv') 
    asthma_df_children = extractChild(asthma_df) 
    tmp = asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]]
    asthma_df_children[random_index[0]].loc[random_index[1][0], random_index[1][1]] = np.nan 
    imputerMiss = MissForest(criterion = 'squared_error', verbose=0)
    asthma_MISS = imputerMiss.fit_transform(asthma_df_children[random_index[0]].loc[:, "steps11":"steps21"])
    asthma_MISS = pd.DataFrame(asthma_MISS, columns=["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"])
    differenceMissForest.append(abs(tmp - asthma_MISS.loc[random_index[1][2], random_index[1][1]]))

In [12]:
print(sum(differenceMissForest)/len(differenceMissForest))

# the variance
print(np.var(differenceMissForest))

386.4966666666667
154383.55407688892


Evaluation of the steps with the whole dataframe

In [2]:
# get index of a random element in a dataframe
def get_random_index(df, columns):
    # select a random element that is not Nan
    # take a random column in columns
    random_index = np.random.randint(0, len(df))
    random_column = columns[np.random.randint(0, len(columns))]

    df.loc[random_index, random_column]

    while pd.isna(df.loc[random_index, random_column]):
        random_index = np.random.randint(0, len(df))
        random_column = columns[np.random.randint(0, len(columns))]

    return [random_index, random_column]

In [6]:
asthma_df = pd.read_csv('asthma_tmp_steps.csv')

adress = []
for i in range(2500):
    adress.append(get_random_index(asthma_df, ["steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21"]))

print(adress)

[[2610, 'steps14'], [2540, 'steps19'], [1968, 'steps19'], [1979, 'steps15'], [273, 'steps11'], [1215, 'steps13'], [1082, 'steps16'], [1832, 'steps17'], [1992, 'steps14'], [139, 'steps18'], [201, 'steps15'], [1747, 'steps12'], [27, 'steps14'], [2564, 'steps14'], [2453, 'steps15'], [1598, 'steps19'], [1677, 'steps11'], [1332, 'steps18'], [1724, 'steps17'], [1160, 'steps20'], [60, 'steps15'], [888, 'steps16'], [1843, 'steps16'], [1580, 'steps21'], [2132, 'steps11'], [1019, 'steps16'], [39, 'steps14'], [1776, 'steps13'], [1772, 'steps21'], [72, 'steps11'], [1257, 'steps15'], [2226, 'steps12'], [2554, 'steps20'], [932, 'steps21'], [188, 'steps11'], [1192, 'steps21'], [1964, 'steps18'], [991, 'steps17'], [1007, 'steps11'], [1063, 'steps21'], [494, 'steps19'], [1232, 'steps17'], [256, 'steps17'], [2538, 'steps11'], [2424, 'steps18'], [2231, 'steps20'], [324, 'steps21'], [1457, 'steps11'], [1473, 'steps12'], [1887, 'steps13'], [2516, 'steps18'], [1764, 'steps16'], [2391, 'steps15'], [2000, 'st

In [7]:
asthma_df.loc[adress[0][0], adress[0][1]]

2157.0

In [10]:
# Play with the number of neighbors
# Look at the kids with the fewest missing data

# TODO: read the csv all the time.

differenceKNN = []
differenceMICE = []
differenceMissForest = []
for random_index in tqdm.tqdm(adress):
    asthma_df = pd.read_csv('asthma_tmp.csv')
    tmp = asthma_df.loc[random_index[0], random_index[1]]
    asthma_df.loc[random_index[0], random_index[1]] = np.nan
    imputerKNN = KNNImputer(n_neighbors = 5)          # n_neighbors=2
    # asthma_df.loc[:, "HR00":"HR23"] = imputerKNN.fit_transform(asthma_df.loc[:, "HR00":"HR23"])
    asthma_KNN = (imputerKNN.fit_transform(asthma_df.loc[:, "steps00":"steps23"]))
    asthma_KNN = pd.DataFrame(asthma_KNN, columns=["steps00", "steps01","steps02","steps03","steps04","steps05","steps06","steps07",
                                                   "steps08","steps09","steps10","steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21","steps22","steps23"])
    # get the absolute value of the difference between the original value and the imputed value
    differenceKNN.append(abs(tmp - asthma_KNN.loc[random_index[0], random_index[1]]))

    asthma_df = pd.read_csv('asthma_tmp.csv')
    tmp = asthma_df.loc[random_index[0], random_index[1]]
    asthma_df.loc[random_index[0], random_index[1]] = np.nan
    mice_imputer = IterativeImputer(verbose=0)
    imputed_data = mice_imputer.fit_transform(asthma_df.loc[:, "steps00":"steps23"])
    # Convert the imputed data array back to a DataFrame
    asthma_MICE = pd.DataFrame(imputed_data, columns=["steps00", "steps01","steps02","steps03","steps04","steps05","steps06","steps07",
                                                   "steps08","steps09","steps10","steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21","steps22","steps23"])
    differenceMICE.append(abs(tmp - asthma_MICE.loc[random_index[0], random_index[1]]))

    


100%|██████████| 2500/2500 [54:11<00:00,  1.30s/it]


In [11]:
print(sum(differenceKNN)/len(differenceKNN))
print(sum(differenceMICE)/len(differenceMICE))

# the variance
print(np.var(differenceKNN))
print(np.var(differenceMICE))

356.5206600137582
340.05754091092905
216104.20991888808
146203.528975811


In [ ]:
for random_index in tqdm.tqdm(adress[:300]): 
    asthma_df = pd.read_csv('asthma_tmp_steps.csv') 
    tmp = asthma_df.loc[random_index[0], random_index[1]]
    asthma_df.loc[random_index[0], random_index[1]] = np.nan 
    imputerMiss = MissForest(criterion = 'squared_error', verbose=0)
    asthma_MISS = imputerMiss.fit_transform(asthma_df.loc[:, "steps00":"steps23"])
    asthma_MISS = pd.DataFrame(asthma_MISS, columns=["steps00", "steps01","steps02","steps03","steps04","steps05","steps06","steps07",
                                                   "steps08","steps09","steps10","steps11","steps12","steps13","steps14","steps15",
                                                   "steps16","steps17","steps18","steps19","steps20","steps21","steps22","steps23"])
    differenceMissForest.append(abs(tmp - asthma_MISS.loc[random_index[0], random_index[1]]))

In [29]:
print(sum(differenceMissForest)/len(differenceMissForest))

# the variance
print(np.var(differenceMissForest))

89.6235533333333
36512.69580669298


In [ ]:
89.6235533333333
36512.69580669298